# 1. Setup

## 1.1 Importing Libraries

In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: omaremaduldeen (omaremaduldeen-German International University - GIU). Use `wandb login --relogin` to force relogin


True

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import os
import ast
from tqdm import tqdm
import random

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import umap.umap_ as umap

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix

## 1.2 Hyperparameters & Configuration

### Hyperparameters Config

In [ ]:
config = dict(

    # Encoder Training
    epochs=25,
    batch_size=32,
    num_workers=1,
    learning_rate=0.001,
    weight_decay = 1e-4,
    temperature = 0.1,
    dropout_rate = 0.1,

    # Early Stopping
    patience = 10,
    delta = 0.01,
    
    # Classification head
    mlp_epochs = 10,
    projection_dim = 128,
    negative_samples_count = 2,
    classifier_output_dim = 2,
    classifier_hidden_dim = 1024,
    classifier_dropout = 0.05,
    classifier_learning_rate = 0.001,

    # Data
    dataset_folder='/Users/mero/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/Ulm/Datasets/small_set',
    dataset_name="reframing_dataset.csv",
    text_model="all-mpnet-base-v2",
    input_dim = 768,
    downloaded = True,
    
    best_model_path ='models/best_model.pth',
    best_mlp_path ='models/best_mlp.pth',
    )

### Reproducibility & Device

In [ ]:
# Reproducibility
torch.use_deterministic_algorithms(True, warn_only=True)
torch.manual_seed(42)
np.random.seed(42)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(42)

# Device Specification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)